In [ ]:
import os
import cv2
from ultralytics import YOLO
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict

# Path to image folders and model
base_dir = "Test_Images"
model_path = "models/yolov11_screwtype_trained_V1.pt"

# Classes (must match your training)
class_names = ['Flat Head Socket Screw', 'Sheet Metal Screw', 'Socket Head Cap Screw', 'Wood Screw']
class_to_idx = {name: i for i, name in enumerate(class_names)}

# Load YOLO model
model = YOLO(model_path)

y_true = []
y_pred = []

# Trackers
no_prediction_count = defaultdict(int)
max_conf_per_class = defaultdict(float)

# Loop over folders
for class_folder in class_names:
    folder_path = os.path.join(base_dir, class_folder)
    for filename in os.listdir(folder_path):
        if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
            img_path = os.path.join(folder_path, filename)
            img = cv2.imread(img_path)

            true_class_idx = class_to_idx[class_folder]  # ✅ FIXED: moved this before usage

            # Inference
            results = model(img, verbose=False)
            boxes = results[0].boxes

            if boxes is not None and len(boxes) > 0:
                # Assume the most confident prediction is the main class
                best_box = boxes[boxes.conf.argmax()]
                predicted_class_idx = int(best_box.cls.item())
                confidence = best_box.conf.item()

                # Update max confidence for correctly predicted class
                if predicted_class_idx == true_class_idx:
                    max_conf_per_class[class_names[true_class_idx]] = max(
                        max_conf_per_class[class_names[true_class_idx]], confidence)
            else:
                predicted_class_idx = -1  # No detection
                no_prediction_count[class_folder] += 1

            y_true.append(true_class_idx)
            y_pred.append(predicted_class_idx)

# Filter out images where no class was predicted
y_true_filtered = [yt for yt, yp in zip(y_true, y_pred) if yp != -1]
y_pred_filtered = [yp for yp in y_pred if yp != -1]

# Evaluation
print("Classification Report:")
print(classification_report(y_true_filtered, y_pred_filtered, target_names=class_names))

# Confusion Matrix
cm = confusion_matrix(y_true_filtered, y_pred_filtered)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.tight_layout()
plt.show()

# Print info
print("\n--- No Prediction Count per Class ---")
for cls, count in no_prediction_count.items():
    print(f"{cls}: {count} image(s) with no detection")

print("\n--- Max Confidence per Correct Class Prediction ---")
for cls in class_names:
    conf = max_conf_per_class.get(cls, 0.0)
    print(f"{cls}: {conf:.4f}")
